# Tashkeel Project 

#AraVec Word Embbedding 

####Please Follow this notbook to refernce word embbeding step

https://github.com/bakrianoo/aravec/blob/master/aravec-with-spacy.ipynb

In [ ]:
!pip install gensim spacy nltk

## Cleaning the text  to estimate the embedding vector

In [ ]:
import gensim
import re
import spacy

                                ##        BIG NOTE       ##

# We will not use this cleaning function we implment our cleaning function , thus we didnot use it 
# for volwelized input , we include it here as following up for the refernced notebook (AraVec with spacy) 




# Clean/Normalize Arabic Text
def clean_str(text):
    search = ["أ","إ","آ","ة","_","-","/",".","،"," و "," يا ",'"',"ـ","'","ى","\\",'\n', '\t','&quot;','?','؟','!']
    replace = ["ا","ا","ا","ه"," "," ","","",""," و"," يا","","","","ي","",' ', ' ',' ',' ? ',' ؟ ',' ! ']
    
    #remove tashkeel
    p_tashkeel = re.compile(r'[\u0617-\u061A\u064B-\u0652]')
    text = re.sub(p_tashkeel,"", text)
    
    #remove longation
    p_longation = re.compile(r'(.)\1+')
    subst = r"\1\1"
    text = re.sub(p_longation, subst, text)
    
    text = text.replace('وو', 'و')
    text = text.replace('يي', 'ي')
    text = text.replace('اا', 'ا')
    
    for i in range(0, len(search)):
        text = text.replace(search[i], replace[i])
    
    #trim    
    text = text.strip()

    return text

### We used "tweet_cbow_100" model from AraVec to estimate word vector

In [ ]:
# Download via terminal commands
!wget "https://archive.org/download/aravec2.0/tweet_cbow_100.zip"

                                ## BIG NOTE ##

'''
- we don't follow here AraVec with spacy notebook , because we will not use the our selected pretrained model 
which is "tweet_cbow_100"


'''

--2021-11-20 20:17:50--  https://archive.org/download/aravec2.0/tweet_cbow_100.zip
Resolving archive.org (archive.org)... 207.241.224.2
Connecting to archive.org (archive.org)|207.241.224.2|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://ia903107.us.archive.org/0/items/aravec2.0/tweet_cbow_100.zip [following]
--2021-11-20 20:17:51--  https://ia903107.us.archive.org/0/items/aravec2.0/tweet_cbow_100.zip
Resolving ia903107.us.archive.org (ia903107.us.archive.org)... 207.241.232.147
Connecting to ia903107.us.archive.org (ia903107.us.archive.org)|207.241.232.147|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 253946430 (242M) [application/zip]
Saving to: ‘tweet_cbow_100.zip’

tweet_cbow_100.zip  100%[===================>] 242.18M  1.88MB/s    in 2m 17s  

2021-11-20 20:20:08 (1.76 MB/s) - ‘tweet_cbow_100.zip’ saved [253946430/253946430]



In [ ]:
!unzip "/content/tweet_cbow_100.zip"

Archive:  /content/tweet_cbow_100.zip
  inflating: tweets_cbow_100         
  inflating: tweets_cbow_100.trainables.syn1neg.npy  
  inflating: tweets_cbow_100.wv.vectors.npy  


In [ ]:
# load the AraVec model
model = gensim.models.Word2Vec.load("/content/tweets_cbow_100")
print("We've",len(model.wv.index2word),"vocabularies")

We've 331679 vocabularies


In [ ]:
# make a directory called "spacyModel"
%mkdir spacyModel

In [ ]:
# export the word2vec fomart to the directory
model.wv.save_word2vec_format("./spacyModel/aravec.txt")

In [ ]:
# using `gzip` to compress the .txt file
!gzip ./spacyModel/aravec.txt

In [ ]:
!python -m spacy  init-model ar spacy.aravec.model --vectors-loc ./spacyModel/aravec.txt.gz

✔ Successfully created model
331679it [00:14, 22651.38it/s]
✔ Loaded vectors from spacyModel/aravec.txt.gz
✔ Sucessfully compiled vocab
331867 entries, 331679 vectors


In [ ]:
# load AraVec Spacy model
nlp = spacy.load("./spacy.aravec.model/")

In [ ]:
# Define the preprocessing Class
class Preprocessor:
    def __init__(self, tokenizer, **cfg):
        self.tokenizer = tokenizer

    def __call__(self, text):
        preprocessed = clean_str(text)
        return self.tokenizer(preprocessed)

In [ ]:
# Apply the `Preprocessor` Class
nlp.tokenizer = Preprocessor(nlp.tokenizer)

### Now we can calculate word vector for arabic word 

Please note that "tweets_cbow_100" 's size output is 100 , there are other pretrained models can vary the output size depending on the corpus that the model trained on .

In [ ]:
nlp("قطة").vector

array([ 0.466963  ,  0.11964946,  2.1654284 ,  0.7553664 , -1.3066205 ,
       -3.372294  , -1.1762583 , -2.8573835 , -0.8307226 , -0.08351412,
        1.4643288 ,  0.09544692,  2.311304  ,  1.49703   , -3.1136436 ,
       -0.69648623, -1.0478779 , -2.2035904 , -2.0805647 ,  3.8258064 ,
       -0.63590115,  1.9480709 ,  1.8645115 , -0.44009277, -1.3259087 ,
       -0.21235421,  1.2734525 ,  0.17556097, -2.156691  ,  1.0667509 ,
        0.99593955,  1.1989368 ,  0.918339  , -0.20591508, -0.3545398 ,
       -1.0034966 ,  0.9294279 , -2.648237  , -2.8392222 , -2.5833333 ,
       -0.6668234 , -0.37400967, -2.526247  ,  0.72259814, -2.495085  ,
       -0.3539619 , -1.0561675 , -2.0031786 ,  0.54362005, -2.7470937 ,
       -1.5453217 ,  0.9318055 , -0.14287415, -0.8798418 , -0.84945273,
       -0.6337022 ,  0.6706522 ,  1.1911404 ,  2.474345  , -2.9734492 ,
       -3.4404132 ,  0.94080114, -3.563805  , -0.07763401, -1.3103951 ,
        1.184813  , -1.5653925 ,  0.76804304,  0.15939131,  0.94

### BIG NOTE 

##### we will use nlp(text).vector below after reading and cleaning the text , we here just call it to be ready to use .

# Preprocessing

## Importing  Libraries 

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import string
import re
import nltk
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix,accuracy_score, classification_report


In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
!pip install -U -q scipy

     |████████████████████████████████| 38.2 MB 19 kB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


# Reading the dataset

In [ ]:
from scipy import io
v = io.loadmat('traindb.mat')

# Split to v(vowelized) text and u(un-vowelized) text

In [ ]:
vtxt_cell = [[row.flat[0] for row in line] for line in v['vtxt_cell'][0]]
uvtxt_cell = [[row.flat[0] for row in line] for line in v['uvtxt_cell'][0]]

# Tokenization 

In [ ]:
each_sentence_tokenaized = []
for each_vtxr in vtxt_cell:
  each_vtxr= str(each_vtxr)
  w = nltk.word_tokenize(each_vtxr)
  each_sentence_tokenaized.append(w)

# Removing Punctuations 

In [ ]:
arabic_punctuations = '''`÷×؛<>_()*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ'''
english_punctuations = string.punctuation
punctuations_list = arabic_punctuations + english_punctuations

In [ ]:
w_nopun = str.maketrans('', '', punctuations_list) #remove punctuationa

In [ ]:
each_sentence_tokenaized_updated = []
for sentence in each_sentence_tokenaized:
  words=[]
  for each_word in sentence:
    preprocessed_word=str(each_word).translate(w_nopun)
    words.append(preprocessed_word)
  each_sentence_tokenaized_updated.append(words)

#Removing non arabic symbols

In [ ]:
#remove_non_arabic_symbols fct
final_version_Sentences_preprocessed = []
for sentence in each_sentence_tokenaized_updated:
  wordssssss = []
  for each_word in sentence:
    ch = re.sub(r'[^\u0600-\u06FF]','', each_word)
    wordssssss.append(ch)		
  final_version_Sentences_preprocessed.append(wordssssss)

# Another Cleaning Step ☹

In [ ]:
for sentence in final_version_Sentences_preprocessed:
  for word in sentence:
    if word == "":
      sentence.remove(word)
      final_version_Sentences_preprocessed.append(sentence)


print(len(final_version_Sentences_preprocessed))
print(final_version_Sentences_preprocessed[0])

12826
['لِلِاسْتِشْرَاقِ', 'أَهْدَافٌ', 'كَثِيرَةٌ', 'مِنْهَا', 'اَلدِّينِيَّةُ', 'وَالِاسْتِعْمَارِيَّةُ', 'وَالْعِلْمِيَّةُ', 'وَالِاقْتِصَادِيَّةُ', 'وَالتِّجَارِيَّةُ', 'وَسَوْفَ', 'نُلْقِي', 'ضَوْءً', 'سَرِيعً', 'عَلَى', 'هَذِهِ', 'الْأَهْدَافِ']


# Appending each tokenized word in one array 

In [ ]:
qqq = []

for sentence in final_version_Sentences_preprocessed:
  for word in sentence:
    qqq.append(word)

# Same Staps applied for u text (un-vowelized ) text 

In [ ]:
uvtxt_cell

[['\ufeffللاستشراق أهداف كثيرة منها: الدينية، والاستعمارية، والعلمية، والاقتصادية، والتجارية. وسوف نلقي ضوء سريع على هذه الأهداف.\n'],
 ['أول: الأهداف الدينية:\n'],
 ['لقد أذهل النصارى المد الإسلامي السريع للإسلام، والذي زحف على المعاقل المسيحية وطرق أبوابها، ومن هنا كانت البابوية في روما في مواجهة هذا الزحف عن طريق الاستشراق، فدفعت بالرهبان والقسس إلى دراسة اللغة العربية والعلوم الإسلامية لكي يحققوا غايتين:\n'],
 ['الأولى: انتزاع مقومات الفكر الإسلامي، وذلك بالتشكيك فيه وإثارة الشبهات حوله، كوسيلة لفرض الثقافة الغربية التي تحاول تطويق الثقافة الإسلامية وصهرها في بوتقة الثقافة الغربية.\n'],
 ['الثانية: محاولة إسقاط النفوذ الإسلامي وتطويقه حتى لا ينتشر في أماكن أخرى من العالم الغربي.\n'],
 ['ثاني: أهداف استعمارية:\n'],
 ['فمنذ فشل أسلوب القوة العسكرية أثناء الحروب الصليبية تبنى الاستعمار حركة الاستشراق واستعان بالمستشرقين واعتبرهم طلائعه الذين يتعرفون على الأفكار ويقومون بالدعايات وإثارة المنازعات وإشعال الخلافات، كما قام المستشرقون بالتجسس على البلاد والتعرف على أحوالها، وكتابة التقاري

In [ ]:
each_sentence_tokenaized_uvtxt_cell = []
for each_utxr in uvtxt_cell:
  each_utxr= str(each_utxr)
  w = nltk.word_tokenize(each_utxr)
  each_sentence_tokenaized_uvtxt_cell.append(w)

In [ ]:
each_sentence_tokenaized_updated_uvtxt_cell = []
for sentence in each_sentence_tokenaized_uvtxt_cell:
  words=[]
  for each_word in sentence:
    preprocessed_word=str(each_word).translate(w_nopun)
    words.append(preprocessed_word)
  each_sentence_tokenaized_updated_uvtxt_cell.append(words)

In [ ]:
#remove_non_arabic_symbols fct
final_version_Sentences_preprocessed_utxt = []
for sentence in each_sentence_tokenaized_updated_uvtxt_cell:
  wordssssss = []
  for each_word in sentence:
    ch = re.sub(r'[^\u0600-\u06FF]', '', each_word)
    wordssssss.append(ch)		
  final_version_Sentences_preprocessed_utxt.append(wordssssss)

In [ ]:
for sentence in final_version_Sentences_preprocessed_utxt:
  for each_word in sentence:
    each_word =str(each_word).strip('[]').split()

In [ ]:
for sentence in final_version_Sentences_preprocessed_utxt:
  for word in sentence:
    if word == "":
      sentence.remove(word)
      final_version_Sentences_preprocessed_utxt.append(sentence)


print(len(final_version_Sentences_preprocessed_utxt))
print(final_version_Sentences_preprocessed_utxt[0])

12826
['للاستشراق', 'أهداف', 'كثيرة', 'منها', 'الدينية', 'والاستعمارية', 'والعلمية', 'والاقتصادية', 'والتجارية', 'وسوف', 'نلقي', 'ضوء', 'سريع', 'على', 'هذه', 'الأهداف']


In [ ]:
ssssssssssssss = []

for sentence in final_version_Sentences_preprocessed_utxt:
  for word in sentence:
    ssssssssssssss.append(word)

In [ ]:
print(len(ssssssssssssss))


1144606


In [ ]:
import numpy 

 Now we loop over each word in v and u array and convert them to word vector to be suitable to input to our model architecture 

In [ ]:
word_vec=[nlp(str(word)).vector for word in ssssssssssssss]

X=numpy.array(word_vec)


"""
Note 

this word embedding calculation for (sssssssssss ) un-vowleized text (this Ahmed Eid 's calling for varibles )
 
"""

## Now Creating Model architecture  

In [ ]:
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Activation

### input size is 100 (one word ) 

we used 300 size(3 input word size )  but this was not effiecient way for input the data to the architecture   

In [ ]:
# define the architecture of the network
model = Sequential()
model.add(Dense(768, input_dim=100,activation="relu"))
model.add(Dense(384, activation="relu", kernel_initializer="uniform"))
model.add(Dense(384, activation="relu", kernel_initializer="uniform"))
model.add(Dense(384, activation="relu", kernel_initializer="uniform"))
model.add(Dense(384, activation="relu", kernel_initializer="uniform"))
model.add(Dense(100))
model.add(Activation("softmax"))

### optmizer is SGD 

In [ ]:
from tensorflow.keras.optimizers import SGD

 one last step to also to convert v(vowelized )text to word embbeding as it will be used as (y_train,y_test) labels or target 

In [ ]:
y_train=[nlp(str(word)).vector for word in qqq]
y_train=numpy.array(y_train)

#### Train test split to create [y_train,y_test,X_train,X_test]

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(word_vec,y_train,test_size = 0.1)

## Fitting the data [X_train , y_train]

The LOSS FUNCTION IS mean_squared_error

number of epochs is 50  

In [ ]:
print("[INFO] compiling model...")
sgd = SGD(lr=0.01)
model.compile(loss="mean_squared_error", optimizer=sgd,
metrics=["accuracy"])
model.fit(X_train,y_train,epochs=50, batch_size=128,verbose=1)

[INFO] compiling model...
Epoch 1/50


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


8943/8943 [==============================] - 192s 21ms/step - loss: 4.6764 - accuracy: 0.0235
Epoch 2/50
8943/8943 [==============================] - 188s 21ms/step - loss: 4.6748 - accuracy: 0.0366
Epoch 3/50
8943/8943 [==============================] - 187s 21ms/step - loss: 4.6536 - accuracy: 0.0668
Epoch 4/50
8943/8943 [==============================] - 186s 21ms/step - loss: 4.6398 - accuracy: 0.0930
Epoch 5/50
8943/8943 [==============================] - 188s 21ms/step - loss: 4.6381 - accuracy: 0.0947
Epoch 6/50
8943/8943 [==============================] - 186s 21ms/step - loss: 4.6378 - accuracy: 0.0949
Epoch 7/50
8943/8943 [==============================] - 188s 21ms/step - loss: 4.6377 - accuracy: 0.0950
Epoch 8/50
8943/8943 [==============================] - 187s 21ms/step - loss: 4.6376 - accuracy: 0.0952
Epoch 9/50
8943/8943 [==============================] - 188s 21ms/step - loss: 4.6376 - accuracy: 0.0953
Epoch 10/50
8943/8943 [==============================] - 188s 21ms

KeyboardInterrupt: ignored

# BIG NOTE 

We noted that the accuracy saturated at epoch number 8 so , we stopped fitting process , because there will not be a big shot increase after this accuracy 